In [ ]:
# Stable Baselines only supports tensorflow 1.x for now
!pip install tensorflow==1.15
!pip install stable-baselines[mpi]==2.10.0

In [ ]:
# Load the TensorBoard notebook extension
%load_ext tensorboard

##  Gym env - Straight and Level

Implementation of Exercise 6: Straight and Level from PPL ground school flight exercises  

Based off of previous code from Exercise 1

In [ ]:
import time
import numpy as np
from random import random

import gym
from gym import spaces

import xpc


MAX_WAIT_TIME = 30 # Time spent waiting for connection until a crash is assumed
ROLL_INCREMENT = 0.01 # Amount to increase/decrease roll
PITCH_INCREMENT = 0.01 # Amount to increase/decrease roll
YAW_INCREMENT = 0.01 # Amount to increase/decrease roll

# Datarefs
roll_dref = "sim/joystick/yoke_roll_ratio" # Yoke roll
roll_electric_dref = "sim/cockpit2/gauges/indicators/roll_electric_deg_pilot" # Roll according to Attitude Indicator

pitch_dref = "sim/joystick/yoke_pitch_ratio" # Yoke pitch
pitch_electric_dref = "sim/cockpit2/gauges/indicators/pitch_electric_deg_pilot" # Pitch according to Attitude Indicator

airspeed_dref = "sim/cockpit2/gauges/indicators/airspeed_kts_pilot" # Indicated airspeed in knots
vertical_speed_dref = "sim/cockpit2/gauges/indicators/vvi_fpm_pilot" # Indicated vertical speed in ft/min

rudder_dref = "sim/cockpit2/controls/rudder_trim" # Rudder trim
heading_dref = "sim/flightmodel/position/hpath" # Heading of plane

altitude_dref = "sim/cockpit2/gauges/indicators/altitude_ft_pilot" # Indicated height, MSL, in feet

throttle_dref = "sim/flightmodel/engine/ENGN_thro" # Throttle level [0...1]

elevation_dref = "sim/flightmodel/position/elevation" # Elevation of plane (ACTUAL, not from an instrument)
test_dref = "sim/test/test_float" # used for testing client connection

reload_situation_dref = "sim/operation/load_situation_1" # Situation for initial conditions of exercise
reload_aircraft_dref = "sim/operation/reload_aircraft_no_art" # reloads aircraft ommiting graphics refresh

class TestEnv(gym.Env): 
    """
    Straight and Level Environment
    Action Space: 
        yoke pitch, yoke roll, rudder trim
    Observation Space: 
        yoke pitch, plane pitch, yoke roll, plane roll, rudder trim, heading difference, 
        altitude, vertical speed, airspeed
    Reward:
        - if within target heading threshold:
            - increase score based on heading
            - if within target pitch threshold, increase based on pitch
            - if within target roll threshold, increase based on roll
    """
    
    def __init__(self,target_pitch, pitch_threshold, target_roll, roll_threshold,
                 target_heading, heading_threshold, max_sim_time, sim_speed):
        super(TestEnv, self).__init__()
        
        # Define action space
        self.action_space = spaces.Box(-1, +1, (3,), dtype=np.float32)
        
        # Define observation space
        self.observation_space = spaces.Box(low=np.array([-1.0, 0.0, -1.0, 0.0, -1.0, 0.0, 0.0, 0.0, 0.0]),
                                            high=np.array([1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0]),
                                            dtype=np.float32, shape = (9,))
                
        # Connect to simulator
        self.client = xpc.XPlaneConnect() # assumes client is online
        self._wait_client(self.client.getDREF, test_dref, 120) # Check if connected
        
        # Local variables
        self.target_pitch = target_pitch
        self.target_roll = target_roll
        self.target_heading = target_heading        
        self.pitch_threshold = pitch_threshold
        self.roll_threshold = roll_threshold
        self.heading_threshold = heading_threshold
        self.max_sim_time = max_sim_time
        self.sim_speed = sim_speed
        self.reward = 0
        
    def reset(self):
        """
        Resets simulator to initial conditions and ensures proper loading time before starting episode
        """
    
        # Reload simulation to begining of exercise
        self.client = xpc.XPlaneConnect()
        self._wait_client(self.client.sendCOMM, reload_situation_dref)
        self._wait_client(self.client.sendCOMM, reload_aircraft_dref)
        
        # TODO: Check if these are still needed; refreshes UDP socket connection
        self.client = xpc.XPlaneConnect()
        self._wait_client(self.client.getDREF, test_dref) # needed
    
        # Set randomized initial pitch, roll, yaw of plane
        # HARDCODED for now
        posi = [37.426654, -122.094514, 1129.226817, -15 + random()*30,     -15 + random()*30,    -15 + random()*30,  1]
        self.client = xpc.XPlaneConnect()
        self._wait_client(self.client.getDREF, test_dref)
        self.client.sendPOSI(posi)
        self._wait_client(self.client.getDREF, test_dref)
        
        # 
        self.start = time.time() # restart timer
        self.reward = 0
        
        
        # Set sim speed
        # Lower grahpic intensity by manually removing shadows, reflections, etc.
        self.client = xpc.XPlaneConnect()
        self._wait_client(self.client.getDREF, test_dref) # needed
        self.client.sendDREFs(
            ["sim/time/sim_speed", 
             "sim/private/controls/perf/disable_shadow_prep",
             "sim/private/controls/caps/use_csm",
             "sim/private/controls/skyc/shadow_level_mount",
             "sim/private/controls/skyc/shadow_offset_mount",
             "sim/private/controls/skyc/shadow_level_hialt",
             "sim/private/controls/skyc/shadow_offset_hialt",
             "sim/private/controls/caps/use_reflective_water"],
            [self.sim_speed, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0])
        
        # TODO: re-implement getting change in observations between timesteps to record 2nd order values
#         # look into this some more, might be causing errors
#         while(self._wait_client(self.client.getDREF, altitude_dref)[0] < 2000):
#             pass # hang while plane indicators reset and simulation reloads
#         self.prev_time = time.time()
#         self.prev_reading = np.array([-1.0, -1.0, -1.0])
#         self.test_time = self._wait_client(self.client.getDREF, "sim/time/total_flight_time_sec")[0]

        return self._next_observation()

    def step(self, action):
        """
        Execute one time step within the environment
        """

        # Take action in simulator
        action = np.clip(action, -1, +1).astype(np.float)
        self._take_action(action) 
        
        # Get obs and reward
        obs = self._next_observation()
        reward, goal = self._get_reward(obs)
        
        self.reward += reward
        
        # Check for terminal states
        done = False
        self.client = xpc.XPlaneConnect()
        if (time.time() - self.start > self.max_sim_time): # Time exceeded
            done = True
        if (self._wait_client(self.client.getDREF, elevation_dref)[0] < 100): # Minimum altitude
            done = True
        if(obs[7] == 0 or obs[7] == 1): # Maximum vertical speed
            done = True

        # Optionally we can pass additional info, we are not using this for now
        info = {}
        
        # Visualizing state during simulation
        self.client.sendTEXT(f"Rew: Tot={self.reward:.1f} Cur={reward:.2f} {goal}\n"  +
                              f"\tPitch: Yoke={obs[0]:.3f} Plane={(obs[1]*360-180):.0f}\n" + 
                              f"\tRoll: Yoke={obs[2]:.3f} Plane={(obs[3]*360-180):.0f}\n" +
                              f"\tYaw: rud={obs[4]:.3f} head_diff={(obs[5]*360 - 180):.0f}\n" +
                              f"\tAlt={(obs[6]*3500+1000):.0f} VSI={(obs[7]*45000-25000):.0f}\n" +
                              f"Act: {action}\n"
                              , -1, -1)
    
        # TODO: time waiting for consistency across sim speed
#         self.test_time2 = self._wait_client(self.client.getDREF, "sim/time/total_flight_time_sec")[0]
#         while(self.test_time2-self.test_time < 0.16): #0.24271 for x6, 0.16716 for x4
#             self.test_time2 = self._wait_client(self.client.getDREF, "sim/time/total_flight_time_sec")[0]
        
#         self.test_time = self.test_time2
        
        return obs, reward, done, info
    
    def _next_observation(self):
        """
        Obtain next observation from simulator
        """
        
        # Collect info through UDP request
        self.client = xpc.XPlaneConnect() # refresh connection
        datarefs = self._wait_client(self.client.getDREFs,
                                     [roll_electric_dref,
                                      pitch_electric_dref,
                                      heading_dref,
                                      rudder_dref,
                                      airspeed_dref,
                                      vertical_speed_dref,
                                      altitude_dref,
                                      throttle_dref,
                                      pitch_dref,
                                      roll_dref])
        
        # Calculate proper roll value
        roll_elec = datarefs[0][0]
        roll_elec = (roll_elec+180)/360 # normalize [-180...180] => [0...360] => [0...1]
        
        # Calculate proper pitch value
        pitch_elec = datarefs[1][0]
        pitch_elec = (pitch_elec+180)/360 # normalize [-180...180] => [0...360] => [0...1]
        
        # Calculate proper heading value
        current_heading = datarefs[2][0]
        heading_diff = (current_heading - self.target_heading + 540) % 360 - 180 # bound to [-180...180]
        heading_diff = (heading_diff+180)/360 # normalize [-180...180] => [0...360] => [0...1]

        
#         # TODO: Absolute differences 
#         roll_elec = datarefs[0][0]
#         roll_elec = abs(roll_elec)/180 # normalize [-180...180] => [0...1]
        
#         pitch_elec = datarefs[1][0]
#         pitch_elec = abs(pitch_elec)/180 # normalize [-180...180] => [0...1]
        
#         current_heading = datarefs[2][0]
#         heading_diff = (current_heading - self.target_heading + 540) % 360 - 180 # bound to [-180...180]
#         heading_diff = abs(heading_diff)/180 # get absolute difference => [0...180] => [0...1]
        
        # Other feature collection
        self.yaw = datarefs[3][0]
        airspeed = np.clip(datarefs[4][0]/150, 0, 1)
        
        # hardcoded expected max and min values found through simulator flight conditions
        vertical_speed = (datarefs[5][0]+25000)/45000 # normalize [-25000...20000] => [0...1]
        altitude = np.clip((datarefs[6][0]-1000)/3500, 0.0, 1.0) # normalize [1000...4500] => [0...1]
        vertical_speed, altitude = np.clip([vertical_speed, altitude], 0, 1)

        throttle = datarefs[7][0]
        self.pitch = datarefs[8][0]
        self.roll = datarefs[9][0]
    
        # TODO: get velocities of each action feature
#         self.next_time = time.time()
#         self.next_reading = np.array([datarefs[0][0]+180, datarefs[1][0]+180, datarefs[2][0]]) # shift all to [0...360]
#         if(all(self.prev_reading == -1.0)): self.prev_reading = self.next_reading
#         delta_readings = np.clip(((self.next_reading-self.prev_reading +540) % 360 - 180)/(self.next_time/self.prev_time), -1.5, 1.5)/1.5 #clip to -1.5, 1.5 then scale to -1...1
#         self.prev_time = self.next_time
#         self.prev_reading = self.next_reading
        
        obs = np.array([self.pitch, 
                        pitch_elec,
                        self.roll,
                        roll_elec,
                        self.yaw,
                        heading_diff,
                        altitude,
                        vertical_speed, 
                        #delta_readings[0], delta_readings[1], delta_readings[2], 
                        airspeed],
                        dtype=np.float32)
        return obs
    
    def _get_reward(self, obs):
        """
        Calculate reward function
        """
        
        pitch_reward = 0
        roll_reward = 0
        heading_reward = 0
        #pitch_vel_discount = 0
        #roll_vel_discount = 0
        #heading_vel_discount = 0
        reward = 0
        goal = ""
        
        # Null reward everywhere except: 
        if obs[1] > (-self.pitch_threshold+180)/360 and obs[1] < (self.pitch_threshold+180)/360:
            goal += "P" # within target pitch threshold
            
        if obs[3] > (-self.roll_threshold+180)/360 and obs[3] < (self.roll_threshold+180)/360:
            goal += "R" # within target roll threshold
            
        if obs[5] > (-self.heading_threshold+180)/360 and obs[5] < (self.heading_threshold+180)/360:
            heading_reward = 1 - pow(abs(obs[5]*360-180)/self.heading_threshold, 0.4)
            #heading_vel_discount = pow(1 - max(obs[10], 0.05), 1/max(obs[5], 0.05))
            goal += "H" # within target heading threshold
            
        # Additional reward when combinations of goals are met
        if(goal == "PH" or goal == "PRH"):
            pitch_reward = 1 - pow(abs(obs[1]*360-180)/self.pitch_threshold, 0.4)
            #pitch_vel_discount = pow(1 - max(obs[8], 0.05), 1/max(obs[1], 0.05))
          
        if(goal == "RH" or goal == "PRH"):
            roll_reward = 1 - pow(abs(obs[3]*360-180)/self.roll_threshold, 0.4)
            #roll_vel_discount = pow(1 - max(obs[9], 0.05), 1/max(obs[3], 0.05))
         
        # Calculate reward
        #reward = pitch_reward*pitch_vel_discount + heading_reward*heading_vel_discount + roll_reward*roll_vel_discount
        reward = pitch_reward + roll_reward + heading_reward
        return reward, goal
    
    
    def _take_action(self, action):
        """
        X-Plane function calls to perform action through UDP requests
        """
        
        # Get current input state from sim (incase of manual intervention)
        self.client = xpc.XPlaneConnect()
        datarefs = self._wait_client(self.client.getDREFs,
                                     [pitch_dref,
                                      roll_dref,
                                      rudder_dref])
        
        # Come up with new control input values
        # For now, overwrite with agents action. Will add in smoothing (limit to movement in time step)
        pitch_diff = action[0] - datarefs[0]
        roll_diff = action[1] - datarefs[1]
        yaw_diff = action[2] - datarefs[2]
        
        pitch_diff, roll_diff, yaw_diff = np.clip([pitch_diff, roll_diff, yaw_diff], -0.25, 0.25) # Bound to min,max change
        self.pitch = datarefs[0] + pitch_diff
        self.roll = datarefs[0] + roll_diff
        self.yaw = datarefs[0] + yaw_diff

        # Send action to sim over UDP
        self._wait_client(self.client.getDREF, test_dref) # needed to ensure UDP order
        self.client.sendDREFs([pitch_dref, roll_dref, rudder_dref],
                             [self.pitch, self.roll, self.yaw])
    
    def _wait_client(self, func, args, wait_time=MAX_WAIT_TIME):
        """
        Request info from client and wait MAX_WAIT_TIME until timing out
        
        Used for testing if client is still responding, and for waiting for blocking functions to finish (ex: loading screen)
        """
        start = None
        result = None
        while result is None:
            try:
                # connect
                result = func(args)
#                 print("\n Connection received!")
                return result
            except:
                if(start is None):
                    start = time.time()
#                     print("Error establishing connection to X-Plane. Trying Again.")
                    pass
                if(time.time() - start > wait_time):
#                     print("Operation timed out.")
                    result = -1
                    raise ValueError("Client Timeout")
                    
                # Start timer and only reset once connection is established.
                # If timer exceeds wait limit, save model, cut connection and restart
                pass
    
    def render(self, mode='human'):
        pass
    
    def close(self):
        pass
    

### Train Model

Run agent and save intermittently in case of X-Plane crash

In [ ]:
import psutil
import os

from stable_baselines import DQN, PPO2, A2C, ACKTR, SAC
from stable_baselines.common.cmd_util import make_vec_env

# Trial configuration
trial_name = 'SAC_trial_1'
trial_dir = "./trials/"
load_trial = False

# Configure model
env = TestEnv(180, 10, 180, 10, 0, 5, 30, 6.0) # Set environment parameters
env = make_vec_env(lambda: env, n_envs=1)
if load_trial:
    timesteps = 580000 # timestep to load from
    model = SAC.load(f"{trial_dir + trial_name}_{timesteps}",
                     env=env, tensorboard_log=f"./log/{trial_name}") 
else:
    timesteps = 0
    model = SAC(policy='MlpPolicy',
                env=env,
                verbose=1,
                learning_rate=3e-4,
                buffer_size=1000000,
                batch_size=256,
                ent_coef='auto',
                train_freq=1,
                gradient_steps=1,
                learning_starts=1000,
                tensorboard_log=f"./log/{trial_name}/"
               ) # Model parameters based from HalfCheetah-v2 reccommendations by gym

# Begin training with error recovery    
model.save(f"{trial_dir + trial_name}_{timesteps}")
start = time.time()

for i in range (30):
    try:
        # Train the agent
        model.learn(20000)
        timesteps += 20000
        model.save(f"{trial_dir + trial_name}_{timesteps}")
    except ValueError:
        # Simulator unresponsive
        print("Error Caught!")
        PROCNAME = "X-Plane.exe"
        for proc in psutil.process_iter():
            # check whether the process name matches
            if proc.name() == PROCNAME:
                proc.kill()
        os.startfile("C:\X-Plane 11\X-Plane.exe")
        
        # Reinstantiate the env and load last saved model
        env = TestEnv(180, 10, 180, 10, 0, 5, 30, 6.0) 
        env = make_vec_env(lambda: env, n_envs=1)
        model = SAC.load(f"{trial_dir + trial_name}_{timesteps}", env, verbose=1, tensorboard_log=f"./log/{trial_name}/")
    
end = time.time()
print("Total Training Time: ", end - start)

# Close simulator
PROCNAME = "X-Plane.exe"
for proc in psutil.process_iter():
    # check whether the process name matches
    if proc.name() == PROCNAME:
        proc.kill()

### Test the Model

Tests the trained agent

In [ ]:
from stable_baselines import DQN, PPO2, A2C, ACKTR, SAC
from stable_baselines.common.cmd_util import make_vec_env

# Configure environment
env = TestEnv(180, 10, 180, 10, 0, 5, 30, 6.0) 
env = make_vec_env(lambda: env, n_envs=1)

# Load model
model = SAC.load("./trials/SAC_trial_1_600000", env)
obs = env.reset()

# Test loaded model
while True:
    action, _states = model.predict(obs, deterministic=True)
    obs, rewards, dones, info = env.step(action)
    env.render()